In [5]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [6]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [12]:
list(train.columns.values)

['id',
 'comment_text',
 'toxic',
 'severe_toxic',
 'obscene',
 'threat',
 'insult',
 'identity_hate']

In [87]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [88]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [ ]:
#Question: why are we stacking char and word?
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [ ]:
count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(train_text)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
scores = []
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))


Multinomial NB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB().fit(X_train_tfidf, train['toxic'])
nb_scores = np.mean(cross_val_score(nb_clf, train_features, train['toxic'], cv=3,scoring='roc_auc'))
nb_scores

Logistic Regression

In [90]:
#note here that without stacking the char td-idf and word td-idf,
#using word only yielded ~.96 accuracy score
from sklearn.linear_model import LogisticRegression
log_clf = LogisticRegression(solver='sag').fit(X_train_tfidf, train['toxic'])
logistic_score = np.mean(cross_val_score(log_clf, train_features, train['toxic'], cv=3, scoring='roc_auc'))
logistic_score

0.9783247934366264

SVM

In [10]:
from sklearn import svm
svm_clf = LogisticRegression().fit(X_train_tfidf, train['toxic'])
svm_score = np.mean(cross_val_score(svm_clf, train_features, train['toxic'], cv=3, scoring='roc_auc'))
svm_score

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

Decision Tree

In [ ]:
from sklearn import tree
dt_clf = tree.DecisionTreeClassifier().fit(X_train_tfidf, train['toxic'])
dt_score = np.mean(cross_val_score(dt_clf, train_features, train['toxic'], cv=3, scoring='roc_auc'))
dt_score

Random Forest